In [28]:
from tqdm import tqdm 
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
def get_par_papers(award_id):
    url = f"https://par.nsf.gov/search/term:{award_id}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        papers = []
        for article_div in soup.select("div.article.item.document"):
            # Title
            title_tag = article_div.select_one("div.title a[itemprop='url'] span[itemprop='name']")
            title = title_tag.get_text(strip=True) if title_tag else None

            # Raw paper link
            link_tag = article_div.select_one("div.title a[itemprop='url']")
            raw_link = link_tag["href"] if link_tag and "href" in link_tag.attrs else None

            # Extract just /biblio/xxxxx part
            match = re.search(r"/biblio/\d+", raw_link if raw_link else "")
            clean_link = "https://par.nsf.gov" + match.group(0) if match else None

            # DOI
            doi_tag = article_div.select_one("div.title + div a[href*='doi.org']")
            doi = doi_tag["href"] if doi_tag and "href" in doi_tag.attrs else None

            # Authors
            authors_tags = article_div.select("div.metadata span.authors span.author[itemprop='author']")
            authors = "; ".join([a.get_text(strip=True) for a in authors_tags]) if authors_tags else None

            # Date published
            date_tag = article_div.select_one("div.metadata span.year time[itemprop='datePublished']")
            date_published = date_tag.get_text(strip=True) if date_tag else None

            # Journal name
            journal = None
            year_span = article_div.select_one("div.metadata span.year")
            if year_span:
                text = year_span.get_text(separator=" ", strip=True)
                match = re.search(r"\((?:.*?)\,\s*(.*?)\)$", text)
                if match:
                    journal = match.group(1)

            papers.append((award_id, title, clean_link, doi, authors, date_published, journal))

        return papers if papers else []

    except Exception as e:
        print(f"Error for award {award_id}: {e}")
        return []


def par(file_path):
    df = pd.read_csv(file_path)
    award_ids = df["awd_id"].astype(str).tolist()

    all_papers = []
    total = len(award_ids)

    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(get_par_papers, aid): aid for aid in award_ids}
        for future in tqdm(as_completed(futures), total=total, desc="🔎 Scraping papers"):
            papers = future.result()
            if papers:
                all_papers.extend(papers)

    papers_df = pd.DataFrame(all_papers, columns=[
        "award_id", "paper_title", "paper_link", "doi", "authors",
        "date_published", "journal_name"
    ])

    # Drop rows with missing or empty paper_title
    papers_df = papers_df.dropna(subset=["paper_title"])
    papers_df = papers_df[papers_df["paper_title"].str.strip() != ""]

    return papers_df



In [24]:
def accuracy(df,origin):

    df_awards = pd.read_csv(origin)

   
    original_award_ids = set(df_awards['awd_id'].astype(str))
    matched_award_ids = set(df['award_id'].astype(str))
    unmatched_award_ids = original_award_ids - matched_award_ids
    results = len(matched_award_ids )/ len(original_award_ids)


    print(f"Total awards in original file: {len(original_award_ids)}")
    print(f"Total awards matched in results: {len(matched_award_ids)}")
    print(f"Awards with no matched papers: {len(unmatched_award_ids)}")
    print(f"rate of found is {results}")
    print()



In [11]:

if __name__ == "__main__":
    start = time.time()
    for year in range(2017,2025):
        print(f" Year: {year}")
        file_name = f"awards/{year}.csv" 

        df = par(file_name)
        df.to_csv(f"PAR/par{year}.csv", index=False)

        print(f"✅ Done in {time.time() - start:.2f} seconds. Total papers: {len(df)}")
        accuracy(df, file_name)


 Year: 2018


C:\Users\admin\AppData\Local\Temp\ipykernel_14136\1733548595.py:57: DtypeWarning: Columns (31,38,41,51,58,61,71,83,111,117,135,145,148,155,158,177,180,183,189,191,193,195,197,199,201,202,203,204,205,206,208,209,216,218,219,223,228,229,234,235,236,237,238,240,241,243,244,245,246,248,249,253,254,255,256,257,258,260,261,263,264,265,266,267,268,270,271,273,274,276,278,279,282,284,285,291,292,293,295,297,299,301,303,305,307,309,310,311,313,315,316,317,319,321,323,325,327,329,331,332,333,334,335,336,337,339,340,342,343,345,346,349,351,352,357,358,363,364,369,370) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
🔎 Scraping papers: 100%|██████████| 12684/12684 [43:29<00:00,  4.86it/s] 


✅ Done in 2618.71 seconds. Total papers: 50042


C:\Users\admin\AppData\Local\Temp\ipykernel_14136\2039591085.py:3: DtypeWarning: Columns (31,38,41,51,58,61,71,83,111,117,135,145,148,155,158,177,180,183,189,191,193,195,197,199,201,202,203,204,205,206,208,209,216,218,219,223,228,229,234,235,236,237,238,240,241,243,244,245,246,248,249,253,254,255,256,257,258,260,261,263,264,265,266,267,268,270,271,273,274,276,278,279,282,284,285,291,292,293,295,297,299,301,303,305,307,309,310,311,313,315,316,317,319,321,323,325,327,329,331,332,333,334,335,336,337,339,340,342,343,345,346,349,351,352,357,358,363,364,369,370) have mixed types. Specify dtype option on import or set low_memory=False.
  df_awards = pd.read_csv(origin)


Total awards in original file: 12684
Total awards matched in results: 8481
Awards with no matched papers: 4203
rate of found is 0.6686376537369915

 Year: 2019


...

C:\Users\admin\AppData\Local\Temp\ipykernel_14136\2039591085.py:3: DtypeWarning: Columns (31,41,53,92,128,134,137,146,149,156,159,166,169,175,179,181,188,190,192,194,196,197,198,199,201,202,203,213,214,215,216,218,219,220,221,222,226,227,232,233,238,239,244,245,250,251,256,257,262,263,265,269,270,271,272,274,275,276,277,283,285,287,289,291,292,293,296,297,299,301,303) have mixed types. Specify dtype option on import or set low_memory=False.
  df_awards = pd.read_csv(origin)


Total awards in original file: 12022
Total awards matched in results: 5022
Awards with no matched papers: 7000
rate of found is 0.4177341540509067

 Year: 2024


C:\Users\admin\AppData\Local\Temp\ipykernel_14136\1733548595.py:57: DtypeWarning: Columns (31,41,53,109,116,119,155,160,163,165,168,169,170,175,183,184,185,186,188,189,190,191,192,193,194,195,196,197,198,199,201,202,203,205,206,207,208,209,210,211,212,216,217,219,220,221,222,224,225,226,227,229,230,231,234,235,236,237,247,249,251,253,257,258,263,264,269,270,275,276,281,282,285,287) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
🔎 Scraping papers: 100%|██████████| 11687/11687 [13:39<00:00, 14.26it/s]


✅ Done in 9177.48 seconds. Total papers: 5965
Total awards in original file: 11687
Total awards matched in results: 2247
Awards with no matched papers: 9440
rate of found is 0.19226490972875845



C:\Users\admin\AppData\Local\Temp\ipykernel_14136\2039591085.py:3: DtypeWarning: Columns (31,41,53,109,116,119,155,160,163,165,168,169,170,175,183,184,185,186,188,189,190,191,192,193,194,195,196,197,198,199,201,202,203,205,206,207,208,209,210,211,212,216,217,219,220,221,222,224,225,226,227,229,230,231,234,235,236,237,247,249,251,253,257,258,263,264,269,270,275,276,281,282,285,287) have mixed types. Specify dtype option on import or set low_memory=False.
  df_awards = pd.read_csv(origin)
